In [1]:
!nvidia-smi

Wed Feb 11 05:54:49 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 580.82.07              Driver Version: 580.82.07      CUDA Version: 13.0     |
+-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   59C    P8             11W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [ ]:
!pip uninstall -y torch torchvision torchaudio mamba-ssm causal-conv1d

!pip install torch==2.7.0 torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126

!pip install https://github.com/Dao-AILab/causal-conv1d/releases/download/v1.6.0/causal_conv1d-1.6.0+cu12torch2.7cxx11abiFALSE-cp312-cp312-linux_x86_64.whl

Found existing installation: torch 2.9.0+cu128
Uninstalling torch-2.9.0+cu128:
  Successfully uninstalled torch-2.9.0+cu128
Found existing installation: torchvision 0.24.0+cu128
Uninstalling torchvision-0.24.0+cu128:
  Successfully uninstalled torchvision-0.24.0+cu128
Found existing installation: torchaudio 2.9.0+cu128
Uninstalling torchaudio-2.9.0+cu128:
  Successfully uninstalled torchaudio-2.9.0+cu128
Looking in indexes: https://download.pytorch.org/whl/cu126
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 215.7 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.7/897.7 kB 124.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.9/8.9 MB 224.9 MB/s eta 0:00:00a 0:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 571.0/571.0 MB 67.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 393.1/393.1 MB 81.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 200.2/200.2 MB 72.6 MB/s eta 0:00:0000:0100:01


In [1]:
!pip install https://github.com/state-spaces/mamba/releases/download/v2.3.0/mamba_ssm-2.3.0+cu12torch2.7cxx11abiFALSE-cp312-cp312-linux_x86_64.whl

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.3/533.3 MB ? eta 0:00:000:00:0100:01


In [ ]:
import os
from google.colab import drive

# 1. Mount the Drive (The boring part)
drive.mount('/content/drive')

# 2. The Magic Shortcut ✨
# This makes your deep folder appear right at /content/project
my_drive_path = '/content/drive/MyDrive/SKRIPSIAHMOMENT_colab'
shortcut_path = '/content/SKRIPSIAHMOMENT_colab'

if not os.path.exists(shortcut_path):
    os.symlink(my_drive_path, shortcut_path)
    print(f"🔗 Linked! Your files are now at: {shortcut_path}")

# 3. Enter the zone
%cd {shortcut_path}
!ls  # Verify you see your files

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/MyDrive/SKRIPSIAHMOMENT_colab
mamba_quin  notebooks+pdf  README.md  sync_to_colab.sh	utils


In [2]:
!git clone https://github.com/Taya-san/SKRIPSIAHMOMENT.git

Cloning into 'SKRIPSIAHMOMENT'...
remote: Enumerating objects: 161, done.
remote: Counting objects: 100% (161/161), done.
remote: Compressing objects: 100% (110/110), done.
remote: Total 161 (delta 97), reused 111 (delta 47), pack-reused 0 (from 0)
Receiving objects: 100% (161/161), 820.41 KiB | 3.48 MiB/s, done.
Resolving deltas: 100% (97/97), done.


In [3]:
%cd SKRIPSIAHMOMENT/
%ls
!git pull

/content/SKRIPSIAHMOMENT
mamba_quin/  notebooks+pdf/  README.md  utils/
Already up to date.


In [ ]:
import torch
from torch import nn
import numpy as np
import transformers
from transformers import AutoModel, AutoConfig, AutoTokenizer, MambaConfig, MambaModel
from datasets import load_dataset
import sklearn.model_selection
import mamba_ssm
try:
    from mamba_ssm.ops.triton.selective_state_update import selective_state_update
    from mamba_ssm.ops.selective_scan_interface import selective_scan_fn, mamba_inner_fn
    
    mamba_ssm.selective_state_update = selective_state_update
    mamba_ssm.selective_scan_fn = selective_scan_fn
    mamba_ssm.mamba_inner_fn = mamba_inner_fn

except ImportError as e:
    print(f"Patch failed! Error: {e}")

In [3]:
from mamba_quin import MambaQuin, MambaQuinConfig
from utils import train_modelnoclt

In [ ]:
pretrained_backbone = MambaModel.from_pretrained("state-spaces/mamba-130m-hf")
pretrained_backbone_config = MambaConfig.from_pretrained("state-spaces/mamba-130m-hf")

config = MambaQuinConfig(
    backbone_config = pretrained_backbone_config.to_dict(),
    num_classes = 2
)

model = AutoModel.from_config(config)
model.backbone.load_state_dict(pretrained_backbone.state_dict())

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:104: UserWarning: 
Error while fetching `HF_TOKEN` secret value from your vault: 'Requesting secret HF_TOKEN timed out. Secrets can only be fetched when running from the Colab UI.'.
You are not authenticated with the Hugging Face Hub in this notebook.
If the error persists, please let us know by opening an issue on GitHub (https://github.com/huggingface/huggingface_hub/issues/new).
  warnings.warn(


Loading weights:   0%|          | 0/242 [00:00<?, ?it/s]

In [5]:
dataset = load_dataset("stanfordnlp/imdb")
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})


In [6]:
tokenizer = AutoTokenizer.from_pretrained("state-spaces/mamba-130m-hf")

In [7]:
train_modelnoclt(model, tokenizer, 1, dataset_dict=dataset, batch_size=8, tr_data=None, ts_data=None)

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Epoch 1/1: 100%|██████████| 3125/3125 [09:24<00:00,  5.54it/s, loss=0.404]

Epoch 1 loss: 0.5419


In [ ]:
from huggingface_hub import login
login(token = "")

In [8]:
model.push_to_hub("GiganticLemon/mambaquin-small")
tokenizer.push_to_hub("GiganticLemon/mambaquin-small")

Writing model shards:   0%|          | 0/1 [00:00<?, ?it/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...qb_8fzi/model.safetensors:   3%|3         | 16.7MB /  520MB            

No files have been modified since last commit. Skipping to prevent empty commit.


CommitInfo(commit_url='https://huggingface.co/GiganticLemon/mambaquin-small/commit/49b93ecede850acf0e568642eff490b17cc2302e', commit_message='Upload tokenizer', commit_description='', oid='49b93ecede850acf0e568642eff490b17cc2302e', pr_url=None, repo_url=RepoUrl('https://huggingface.co/GiganticLemon/mambaquin-small', endpoint='https://huggingface.co', repo_type='model', repo_id='GiganticLemon/mambaquin-small'), pr_revision=None, pr_num=None)

In [9]:
model_test = AutoModel.from_pretrained("GiganticLemon/mambaquin-small",
                                       trust_remote_code = True)
tokenizer_test = AutoModel.from_pretrained("GiganticLemon/mambaquin-small")

config.json:   0%|          | 0.00/340 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/520M [00:00<?, ?B/s]

Loading weights:   0%|          | 0/20 [00:00<?, ?it/s]

AttributeError: 'MambaQuin' object has no attribute 'all_tied_weights_keys'

In [10]:
!nvidia-smi

Mon Feb  9 16:01:57 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   45C    P8             10W /   70W |       2MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----